In [1]:
# Don't need to run this each time notebook is started 
# Reqs already satisfied in kaggle NB.
!pip install tifffile 
!pip install imagecodecs
!pip install torch
!pip install torchvision
!pip install efficientnet_pytorch torchtoolbox
!pip install scikit-build
!pip install blosc

     |████████████████████████████████| 146 kB 3.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 17.9 MB 3.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 748.8 MB 4.3 kB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 5.9 MB 3.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 58 kB 3.1 MB/s eta 0:00:011
     |████████████████████████████████| 17.2 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 995 kB 41.5 MB/s eta 0:00:01
  Created 

In [1]:
from efficientnet_pytorch import EfficientNet

In [2]:
# Imports here
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import os
import random
import math
import skimage.io
#from csv_loader import load_csv

# Tiff visualisation imports and downloads
import numpy as np
#import tifffile as tiff

# For re-importing python modules
import importlib
#importlib.reload(csv_loader.py)



In [3]:
#use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [4]:
pd.read_csv("train.csv")

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0
...,...,...,...,...
10611,ffd2841373b39792ab0c84cccd066e31,radboud,0,negative
10612,ffdc59cd580a1468eac0e6a32dd1ff2d,radboud,5,4+5
10613,ffe06afd66a93258f8fabdef6044e181,radboud,0,negative
10614,ffe236a25d4cbed59438220799920749,radboud,2,3+4


In [5]:
class load_csv(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)# todo remove sample for debug
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
        
    
    def __getitem__(self, index):
        #img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0] +".tiff")
        image = torch.from_numpy(skimage.io.MultiImage(img_path)[2]).permute(2,0,1).float()
        #Image.MAX_IMAGE_PIXELS = None
                
        '''
        train_transform = transforms.Compose([transforms.RandomRotation(30),
                                              transforms.RandomResizedCrop(224),
                                              transforms.ToPILImage(),
                                              transforms.RandomHorizontalFlip(),
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225])])
        '''
        #image.transform = transforms.RandomResizedCrop(224)
        
        isup_grade = int(self.annotations.iloc[index,:]['isup_grade'])
        #label = np.zeros(5).astype(np.float32)
        #y_label = label[:isup_grade] = 1.
        y_label = torch.tensor(isup_grade)
        
        self.transform= transforms.Compose([transforms.ToPILImage(),
                                            transforms.RandomResizedCrop(400), 
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225])])
        if self.transform:
            image = self.transform(image)
        
        return (image, y_label)

In [6]:
file_info = pd.read_csv('train.csv').copy()

In [7]:
sample_size = 10000
first_train_list = file_info.sample(sample_size)
first_train_list 
first_train_list.to_csv("sample_dir/sample.csv", sep=",", index=False)

In [8]:
#dataset = load_csv(csv_file='sample_dir/sample.csv', root_dir='train_images', transform=transforms.ToTensor())
dataset = load_csv(csv_file='sample_dir/sample.csv', root_dir='train_images')

In [9]:
# Creating a subset of 50 sample images to work with will be good to start building a first neural network and debug .TIFF related issues
sample_size = dataset.annotations.shape[0]
train_ratio = .75
valid_ratio = .15
test_ratio = 1-(train_ratio + valid_ratio)
train_size = int(train_ratio*sample_size)
valid_size = int(valid_ratio*sample_size)
test_size = int(test_ratio*sample_size)

# In case there are decimal/rounding errors
if train_size + valid_size + test_size > sample_size:
    train_size-=1
elif train_size + valid_size + test_size < sample_size:
    train_size+=1

# List of sample images with corresponding labels
#image_id = random.sample(list(file_info['image_id']), sample_size)
#y_label = [int(file_info.loc[file_info['image_id'] == x, 'isup_grade']) for x in image_id]
#image_id = [str(x) + '.tiff' for x in image_id]
#sample_imgs = pd.DataFrame({'image_id':image_id, 'y_label':y_label})

In [10]:
[train_size, valid_size, test_size]

[7501, 1500, 999]

In [11]:
len(dataset)

10000

In [12]:
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

In [24]:
# Pass transforms in here, then run the next cell to see how the transforms look
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=False)

In [15]:
# Define model type and freeze parameters
#model = models.vgg16(pretrained=True)
# for efficientnet self.arch._fc = nn.Linear(in_features=1280, out_features=500, bias=True)
model = EfficientNet.from_pretrained('efficientnet-b1',num_classes=6)
if torch.cuda.is_available():
    model.cuda()

num_ftrs = model._fc.in_features
    
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True    
#for param in model.parameters():
#    param.requires_grad = False

Loaded pretrained weights for efficientnet-b1


In [16]:
# Defining Squential Classifier transformation with high drop out rate
# classifier = nn.Sequential(
#     nn.Linear(in_features=1280, out_features=500, bias=True),
#     #nn.Linear(224, 112, bias=True),
#                            nn.ReLU(),
#                            nn.Dropout(.5),
# #                            nn.Linear(112, 66, bias=True),
# #                            nn.ReLU(),
# #                            nn.Dropout(.5),
#                            nn.Linear(112, 1, bias=True))
    
# model.classifier = classifier
model.to(device);

In [15]:
for param in model.parameters():
    print(param.shape)

torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([32])
torch.Size([32, 1, 3, 3])
torch.Size([32])
torch.Size([32])
torch.Size([8, 32, 1, 1])
torch.Size([8])
torch.Size([32, 8, 1, 1])
torch.Size([32])
torch.Size([16, 32, 1, 1])
torch.Size([16])
torch.Size([16])
torch.Size([16, 1, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([4, 16, 1, 1])
torch.Size([4])
torch.Size([16, 4, 1, 1])
torch.Size([16])
torch.Size([16, 16, 1, 1])
torch.Size([16])
torch.Size([16])
torch.Size([96, 16, 1, 1])
torch.Size([96])
torch.Size([96])
torch.Size([96, 1, 3, 3])
torch.Size([96])
torch.Size([96])
torch.Size([4, 96, 1, 1])
torch.Size([4])
torch.Size([96, 4, 1, 1])
torch.Size([96])
torch.Size([24, 96, 1, 1])
torch.Size([24])
torch.Size([24])
torch.Size([144, 24, 1, 1])
torch.Size([144])
torch.Size([144])
torch.Size([144, 1, 3, 3])
torch.Size([144])
torch.Size([144])
torch.Size([6, 144, 1, 1])
torch.Size([6])
torch.Size([144, 6, 1, 1])
torch.Size([144])
torch.Size([24, 144, 1, 1])
torch.Size([24])
t

In [20]:
criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
print_every = 20


In [21]:
def validate_data_function(model, test_loader, criterion):
    test_loss = 0
    accuracy = 0
    
    for ii, (inputs, labels) in enumerate(test_loader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        test_loss += criterion(output, labels.long()).item()
        
        ps = torch.exp(output)
        equality = (labels == ps.argmax(dim=1))
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [22]:
import time
def train_test(epochs, model, train_loader, device, optimizer, criterion, print_every, valid_loader):
    start = time.time()

    test_loss = 0
    steps = 0
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        
        for ii, (inputs, labels) in enumerate(train_loader):
            steps += 1
        
            inputs, labels = inputs.to(device), labels.to(device)
        
            optimizer.zero_grad()
        
       
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
        
            if steps % print_every == 0:
                model.eval()

                with torch.no_grad():
                    valid_loss, accuracy = validate_data_function(model, valid_loader, criterion)
            
                print(f"Epoch {epoch+1}/{epochs}..| "
                      f"Train loss: {running_loss/print_every:.3f}..| "
                      f"Validation loss: {valid_loss/print_every:.3f}..| "                  
                      f"Validation accuracy: {accuracy/len(valid_loader):.3f}|")
            
                running_loss = 0
                model.train()

    end = time.time()
    print(end-start)

## I've built and ran a first version of this NN. Now, as I go about making my first adjustments, I have several decisions I need to make:
Question 1) Which of the three picture qualities do I want to choose from my multi-dimensional .TIFF file?
Preliminary answer/thoughts: Not the highest resolution (dim = 0) because it overloads the kernel's memory (some of these pictures can reach 500MB in size). It would have to be either medium (dim = 1) or low (dim = 2) quality pictures.

Question 2) What should the size of my input be when I feed it into the model?
Preliminary answer/thoughts: This will depend on the answer to Question 1. It will also depend on pre-processing transformations that could be needed to improve prediction scores.

Question 3) Should I train my model with more than 1 folds?
Preliminary answer/thoughts: Yes. The model must be evenly trained by the same training data because, according to Kaggle competition, the model will be evaluated training-data-based testing metric (read project proposal.pdf in project repository).

These questions are designed to promote greater prediction scores while being conscious of training times and system capacities. Further below, I will be recording parameters and results for different tests, and hope to eventually have answers for the questions above.

In [ ]:
# tile approach.

In [26]:
# 1000 sample pics, low quality, 224 randomcrop frame size
train_test(epochs, model, train_loader, device, optimizer, criterion, print_every, valid_loader)
# Result: 51.4s

Epoch 1/10..| Train loss: -25.058..| Validation loss: -1.850..| Validation accuracy: 0.067|
Epoch 2/10..| Train loss: -25.208..| Validation loss: -8.110..| Validation accuracy: 0.000|
Epoch 3/10..| Train loss: -31.776..| Validation loss: -8.674..| Validation accuracy: 0.067|
Epoch 4/10..| Train loss: -34.874..| Validation loss: -10.048..| Validation accuracy: 0.000|
Epoch 5/10..| Train loss: -36.871..| Validation loss: -13.750..| Validation accuracy: 0.200|
Epoch 6/10..| Train loss: -26.755..| Validation loss: -12.898..| Validation accuracy: 0.067|
Epoch 7/10..| Train loss: -46.194..| Validation loss: -5.996..| Validation accuracy: 0.267|
Epoch 8/10..| Train loss: -51.002..| Validation loss: -4.397..| Validation accuracy: 0.267|
Epoch 9/10..| Train loss: -47.457..| Validation loss: -8.642..| Validation accuracy: 0.133|
Epoch 10/10..| Train loss: -44.559..| Validation loss: -12.302..| Validation accuracy: 0.067|
51.41079807281494


In [18]:
# New nn.Entropyloss used instead of NNLoss
# 500 sample pics, low quality, 448 randomcrop frame size
train_test(epochs, model, train_loader, device, optimizer, criterion, print_every, valid_loader)
# Result: 95.3s
# 28% acc

Epoch 1/10..| Train loss: 1.734..| Validation loss: 1.818..| Validation accuracy: 0.240|
Epoch 1/10..| Train loss: 1.826..| Validation loss: 1.832..| Validation accuracy: 0.187|
Epoch 1/10..| Train loss: 1.706..| Validation loss: 1.720..| Validation accuracy: 0.280|
Epoch 1/10..| Train loss: 1.789..| Validation loss: 1.723..| Validation accuracy: 0.293|
Epoch 1/10..| Train loss: 1.665..| Validation loss: 1.730..| Validation accuracy: 0.240|
Epoch 2/10..| Train loss: 1.245..| Validation loss: 1.685..| Validation accuracy: 0.253|
Epoch 2/10..| Train loss: 1.718..| Validation loss: 1.676..| Validation accuracy: 0.253|
Epoch 2/10..| Train loss: 1.705..| Validation loss: 1.707..| Validation accuracy: 0.213|
Epoch 2/10..| Train loss: 1.658..| Validation loss: 1.679..| Validation accuracy: 0.267|
Epoch 2/10..| Train loss: 1.429..| Validation loss: 1.944..| Validation accuracy: 0.320|
Epoch 3/10..| Train loss: 0.819..| Validation loss: 3.428..| Validation accuracy: 0.320|
Epoch 3/10..| Train l

In [67]:
# 1000 sample pics, medium quality, 224 randomcrop frame size
train_test(epochs, model, train_loader, device, optimizer, criterion, print_every, valid_loader)
# Result: 534.9s, more than 10x longer than small picture. There are ~10,000 pictures we will need to train the model on eventually, implying that
# it will take ~53,490s = ~15hrs to train the model on the entire dataset with current parameters, which sounds a bit brutal, especially because
# I'm paying for an AWS instance that costs $1.2/hr
# The low validation accuracy has me worried regarding how the loss criterion is being calculated.

RuntimeError: CUDA out of memory. Tried to allocate 116.00 MiB (GPU 0; 11.17 GiB total capacity; 10.45 GiB already allocated; 93.31 MiB free; 10.69 GiB reserved in total by PyTorch)

In [34]:
# 1000 sample pics, medium quality, 448 randomcrop frame size
train_test(epochs, model, train_loader, device, optimizer, criterion, print_every, valid_loader)
# Result: 52.3s

Epoch 1/10..| Train loss: -0.057..| Validation loss: -0.013..| Validation accuracy: 0.333|


KeyboardInterrupt: 

In [29]:
# 1000 sample pics, medium quality, 448 randomcrop frame size
train_test(epochs, model, train_loader, device, optimizer, criterion, print_every, valid_loader)
# Result: 52.3s

Epoch 1/10..| Train loss: 1.817..| Validation loss: 0.664..| Validation accuracy: 0.250|
Epoch 2/10..| Train loss: 0.628..| Validation loss: 0.736..| Validation accuracy: 0.150|
Epoch 2/10..| Train loss: 1.523..| Validation loss: 0.673..| Validation accuracy: 0.350|
Epoch 3/10..| Train loss: 1.299..| Validation loss: 0.708..| Validation accuracy: 0.250|
Epoch 4/10..| Train loss: 0.286..| Validation loss: 0.756..| Validation accuracy: 0.300|
Epoch 4/10..| Train loss: 1.328..| Validation loss: 0.739..| Validation accuracy: 0.300|
Epoch 5/10..| Train loss: 0.781..| Validation loss: 0.857..| Validation accuracy: 0.100|
Epoch 5/10..| Train loss: 1.255..| Validation loss: 1.065..| Validation accuracy: 0.200|
Epoch 6/10..| Train loss: 1.396..| Validation loss: 0.775..| Validation accuracy: 0.150|
Epoch 7/10..| Train loss: 0.423..| Validation loss: 0.912..| Validation accuracy: 0.100|
Epoch 7/10..| Train loss: 1.325..| Validation loss: 1.090..| Validation accuracy: 0.200|
Epoch 8/10..| Train l

In [ ]:
# 10,000 sample pics, medium quality, 448 randomcrop frame size
train_test(epochs, model, train_loader, device, optimizer, criterion, print_every, valid_loader)
# Result: 52.3s

Epoch 1/10..| Train loss: 1.682..| Validation loss: 14.017..| Validation accuracy: 0.282|
Epoch 1/10..| Train loss: 1.806..| Validation loss: 13.144..| Validation accuracy: 0.250|
Epoch 1/10..| Train loss: 1.729..| Validation loss: 12.920..| Validation accuracy: 0.271|
Epoch 1/10..| Train loss: 1.629..| Validation loss: 12.757..| Validation accuracy: 0.292|
Epoch 1/10..| Train loss: 1.662..| Validation loss: 13.192..| Validation accuracy: 0.267|
Epoch 1/10..| Train loss: 1.686..| Validation loss: 13.192..| Validation accuracy: 0.267|
Epoch 1/10..| Train loss: 1.594..| Validation loss: 13.007..| Validation accuracy: 0.282|
Epoch 1/10..| Train loss: 1.638..| Validation loss: 12.663..| Validation accuracy: 0.303|
Epoch 1/10..| Train loss: 1.701..| Validation loss: 12.768..| Validation accuracy: 0.234|
Epoch 1/10..| Train loss: 1.684..| Validation loss: 12.400..| Validation accuracy: 0.272|
Epoch 1/10..| Train loss: 1.535..| Validation loss: 17.165..| Validation accuracy: 0.270|
Epoch 1/10

In [38]:
!ps -elf | grep python

4 S ec2-user  7222  7108  0  80   0 - 281343 SyS_ep 18:14 ?       00:00:23 /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/jupyter-notebook --notebook-dir=/home/ec2-user/SageMaker/ --ip=0.0.0.0 --NotebookApp.token= --port=8443 --NotebookApp.disable_check_xsrf=True --certfile=/home/ec2-user/.jupyter/notebookcert.pem --keyfile /home/ec2-user/.jupyter/notebookkey.key
4 S 502       7258  7257  0  80   0 - 224085 -     18:14 ?        00:00:00 /home/nbiagent-user/miniconda/bin/python /home/nbiagent-user/miniconda/bin/nbiagent /home/nbiagent-user/nbiagentcert.pem /home/nbiagent-user/nbiagentkey.key
0 S ec2-user 10045  7222  2  80   0 - 21989392 core_s 18:24 ?     00:00:46 /home/ec2-user/anaconda3/envs/python3/bin/python -m ipykernel -f /home/ec2-user/.local/share/jupyter/runtime/kernel-880437f3-bc31-4dbb-8531-8a4f887b0c46.json
0 S ec2-user 18100 10045  0  80   0 - 28289 -      19:01 pts/1    00:00:00 /bin/sh -c ps -elf | grep python

In [23]:
torch.cuda.empty_cache()

In [43]:
torch.cuda.get_device_properties(device).total_memory

11996954624

In [17]:
!pip install GPUtil

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=37d01e19df60684b03270e663dc45330f583a21767e69052ff4b75e1e8edc05e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/79/c1/b2/b6fc2647f693a084da25e1d31328ab3dbb565cc58fea37e973
Successfully built GPUtil
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [58]:
!cuda.collect()

/bin/sh: -c: line 1: syntax error: unexpected end of file


In [35]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 19% |


In [1]:
!nvidia-smi

Mon Aug 17 01:29:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   42C    P0    79W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  